In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Assumes scale images are on your Drive under "Images"

!ls /content/drive/MyDrive/Images/*csv

/content/drive/MyDrive/Images/labels.csv
/content/drive/MyDrive/Images/SalmonFishScalesCoordinates-V2.csv


In [3]:
import os.path
import numpy as np
import math
from matplotlib.image import imread
import matplotlib.pyplot as plt
import skimage.draw as drw

SCALE_WIDTH = 1500
SCALE_HEIGHT = 2000
IMAGE_HEIGHT = 1920
TRANSECT_LENGTH = 750
NUM_FOLDS = 10

PATH=os.sep + os.path.join("content", "drive", "MyDrive", "Images")

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def fill_boarder(gray_img, thickness, fill_value=255):
  gray_img[:thickness,:] = fill_value
  gray_img[-thickness:,:] = fill_value
  gray_img[:,:thickness] = fill_value
  gray_img[:,-thickness:] = fill_value

def scale_edge_detector(x,index, window_size, frequency_factor, orientation):
  if not (orientation == 'LR' or orientation == 'RL'):
    raise Exception("Edge detection orientation must be 'LR' or 'RL'.")

  if window_size % 2 != 1:
    raise Exception("Window size must be an odd number.")

  l_sum = np.sum(x[index - int(window_size / 2): index])
  r_sum = np.sum(x[index + 1: index + int(window_size / 2)])

  if orientation == 'LR':
    if r_sum > l_sum * frequency_factor:
      return True
    else:
      return False
  else:
    if r_sum * frequency_factor > l_sum:
      return True
    else:
      return False

def mask1d(gray_img, kernel_size, threshold=20):
  if kernel_size % 2 != 1:
    raise Exception("Kernel size must be an odd number.")


  mask = np.zeros(gray_img.shape, dtype=np.uint8)
  filter = np.ones(kernel_size) / kernel_size

  for y in range(gray_img.shape[0]):
    delta = np.convolve(gray_img[y,:], filter, mode='same') - gray_img[y,:]
    mask[y, (np.abs(delta) > threshold)] = 255.

  return mask

def mask(gray_img, kernel_size, threshold=20):
  if kernel_size % 2 != 1:
    raise Exception("Kernel size must be an odd number.")

  xd = mask1d(gray_img, 9, 20)
  yd = mask1d(np.transpose(gray_img), 9, 20).transpose()
  mask = xd /2 + yd / 2

  return mask

def denoise(gray_img, kernel_size, percent_activated_threshold=0.4):
  if kernel_size % 2 != 1:
    raise Exception("Kernel size must be an odd number.")

  activation_cutoff = percent_activated_threshold * kernel_size * kernel_size * 255
  denoised_img = gray_img.copy()

  for j in range(int(kernel_size / 2), gray_img.shape[0] - int(kernel_size / 2)):
    for k in range(int(kernel_size / 2), gray_img.shape[1] - int(kernel_size / 2)):
      if gray_img[j, k] > 120:
        if np.sum(gray_img[j:j+kernel_size,k:k+kernel_size]) < activation_cutoff:
          denoised_img[j, k] = 0

  return denoised_img

def detect_scale_contour(gray_img, padding=25):
  contour = []

  for j in range(padding, gray_img.shape[0] - padding):
    k = padding
    k_min = padding

    # Burn in period to make sure we are not detecting a scale on the edge
    while k < gray_img.shape[1] - padding and np.sum(gray_img[j, k-10:k+25]) > 30 * 255:
      k = k + 1

    while k < gray_img.shape[1] - padding:
      if gray_img[j, k] > 0 and scale_edge_detector(gray_img[j,:], k, 25, 5, 'LR'):
        contour.append((j, k))
        k_min = k
        k = gray_img.shape[1] - padding
      k = k + 1

    # Burn in period to make sure we are not detecting a scale on the edge
    while k > k_min and np.sum(gray_img[j, k-25:k+10]) > 30 * 255:
      k = k - 1

    while k > k_min:
      if gray_img[j, k] > 0 and scale_edge_detector(gray_img[j,:], k, 25, 5, 'RL'):
        contour.append((j, k))
        k = 0
      k = k - 1

  return contour

def dilate_contour(contour, thickness):

  dialated_contour = set()
  for (j, k) in contour:
    for l in range(j - thickness, j + thickness):
      for m in range(k - thickness, k + thickness):
        dialated_contour.add((l, m))

  return list(dialated_contour)

def draw_box(gray_img, top_left_point, bottom_right_point, color=255):
  j1, k1 = top_left_point
  j2, k2 = bottom_right_point

  gray_img[j1,k1:k2+1] = color
  gray_img[j2,k1:k2+1] = color
  gray_img[j1:j2+1,k1] = color
  gray_img[j1:j2+1,k2] = color

def draw_line(gray_img, mid_point, end_point, color=255):

  rr, cc = drw.line(*mid_point, *end_point)
  gray_img[rr, cc] = color

def draw_contour(size, contour, color=255):

  img = np.zeros(size, dtype=np.uint8)
  for (j, k) in contour:
    img[j, k] = color

  return img

def load_img_to_gray(file):
  img = imread(file)
  gray = rgb2gray(img)

  return gray

def show_image_and_contour(gray):

  plt.imshow(gray, cmap=plt.get_cmap('gray'))
  plt.show()

  masked_image = mask(gray, 9, 20)
  denoised_mask = denoise(masked_image, 101, .025)

  contour = detect_scale_contour(denoised_mask, 100)
  dilated_contour = dilate_contour(contour, 5)
  drawn_contour = draw_contour(denoised_mask.shape, dilated_contour)

  plt.imshow(np.abs(drawn_contour - 255), cmap=plt.get_cmap('gray'))
  plt.show()

def frame_image_w_mean(gray, j_m, k_m):
  mean_pixel_value = np.mean(gray)

  if j_m - SCALE_HEIGHT // 2 > 0:
    gray[0:j_m - SCALE_HEIGHT // 2,:] = mean_pixel_value
    gray[j_m + SCALE_HEIGHT // 2:,:] = mean_pixel_value

  if k_m - SCALE_WIDTH // 2 > 0:
    gray[:, 0:k_m - SCALE_WIDTH // 2] = mean_pixel_value
    gray[:,k_m + SCALE_WIDTH // 2:] = mean_pixel_value

  #return gray

def calculate_transect_endpoint(mid_point, offset, length):
  """
  Given an offset from a line extending direclty down from the
  mid-point, determine the coordinates of the other endpoint of
  a line that starts at the mid-point and extends in the direction
  of the offset of specified length

  :param mid_point: the coordinates of the mid-point in row-major format
  :param offset: offset angle in radians from 6 o'clock applied counter-clockwise
  :param length: describe about parameter p3
  :return: the coordinates of the endpoint in row-major format
  """
  j, k = mid_point

  return (j + int(length * math.cos(offset)),
          k + int(length * math.sin(offset)))

def calculate_transect_angle(mid_point, end_point):
  """
  Given two points, calculated the angle between a line that
  extends down from the mid-point, opens counter-clockwise
  to the line that is defined by the mid_point and end_point

  :param mid_point: the coordinates of the mid-point in row-major format
  :param end_point: the coordinates of the end-point of the transect
     in row-major format
  :return: angle in radians
  """

  offset = 0

  # if the transect angle is at pi/2 or 3 pi/2, the atan will be undefined
  if abs(mid_point[0] - end_point[0]) < 1e-5:
    if mid_point[0] < end_point[0]:
      offset = math.pi / 2
    else:
      offset = 3 * math.pi / 2

  else:
    offset = math.atan((mid_point[1] - end_point[1]) /
                       (mid_point[0] - end_point[0]))

    # if end point is above mid point, add pi to offset as it would have
    #  been caculated relative to line extending above mid point.
    if(end_point[0] < mid_point[0]):
      offset = offset + math.pi

    # if offset is negative, add to 2pi
    if(end_point[1] < mid_point[1]):
      offset = 2 * math.pi + offset

  return offset

def calculate_coordinates_of_transects(mid_point, transect_angle, offset_step_size, transect_count):
  """
  Determine the coordintates of points along transect lines at specified offsets
  from an initial trasect.

  :param mid_point: the coordinates of the mid-point in row-major format
  :param offset: transect angle in radians from 6 o'clock applied counter-clockwise
  :param offset_step_size: Offset in radians between transects
  :param transect_count: The number of transects to generate in addition to
    main transect
  :return: a list of lists containing the coordinates of points on transects
    extending out from the mid_point.  Indexing of transects alternates out
    from main transect.
  """
  transects_coordinates = []
  calculate_transect_endpoint(mid_point, transect_angle, TRANSECT_LENGTH)
  transects_coordinates.append(drw.line(*mid_point,
                                        *calculate_transect_endpoint(mid_point, transect_angle, TRANSECT_LENGTH))
                               )

  for offset in [o for i in range(1, int(transect_count / 2) + 1) for o in (i * offset_step_size, -i * offset_step_size)]:
    transects_coordinates.append(drw.line(*mid_point,
                                          *calculate_transect_endpoint(mid_point, transect_angle + offset, TRANSECT_LENGTH))
                                 )

  return transects_coordinates

def draw_transects(gray_img, transects_coordinates, color = 250):
  """
  Draws transects on a copy of grayscale image.

  :param gray_img: 2d array of a grayscale image
  :param trasects_coordinates: a list of lists containing the coordinates of points on transects
    extending out from the mid_point.  Indexing of transects alternates out
    from main transect.
  :param color: pixel value for drawn transect
  :return: 2d array representing grayscale image with transects applied
  """
  img_w_transects = np.copy(gray_img)
  j_max = gray_img.shape[0]
  k_max = gray_img.shape[1]

  for i, transect in enumerate(transects_coordinates):
    rr, cc = transect
    rr[rr >= j_max] = j_max - 1
    cc[cc >= k_max] = k_max - 1

    img_w_transects[rr, cc] = color

  return img_w_transects

def extract_transects_signals(gray_img, transects_coordinates):

  transect_signals = []
  for i, transect in enumerate(transects_coordinates):
    signal = []
    rr, cc = transect
    for j, k in zip(rr, cc):
      signal.append(gray_img[j, k])
    transect_signals.append(signal)

  return transect_signals

def convolve_transects_signals(transects_signals, kernel_size):

  filter = np.ones(kernel_size) / kernel_size
  convolved_signals = []

  for signal in transects_signals:
    convolved_signals.append(convolve_transects_signal(signal, kernel_size))

  return convolved_signals

def convolve_transects_signal(transects_signal, kernel_size):

  filter = np.ones(kernel_size) / kernel_size
  signal_length = max(len(transects_signal), kernel_size) - min(len(transects_signal), kernel_size) + 1
  offset = int((len(transects_signal) - signal_length) / 2)
  return(np.convolve(transects_signal, filter, mode='valid') - transects_signal[offset:-offset])


def plot_transect_signals(transects_signals, figure_path, figure_filename):
  n = len(transects)

  fig, axes = plt.subplots(2 * n)
  for i, signal in enumerate(transects_signals):
    axes[i].plot(signal)

  convolved_signals = convolve_transects_signals(transects_signals, 5)
  for i, signal in enumerate(convolved_signals):
    axes[i + n].plot(signal)

  fig.savefig(os.path.join(figure_path, figure_filename +".png"), format="png",
              bbox_inches="tight")

def plot_transect_signals_for_image(filepath, mid_point, transect_angle,
                                    offset_step_size, transect_count,
                                    figure_path):
  gray = load_img_to_gray(filepath)
  filename = os.path.splitext(os.path.basename(filepath))[0]

  transects = calculate_coordinates_of_transects(mid_point, transect_angle,
                                                 offset_step_size, transect_count)
  transects_signals = extract_transects_signals(gray, transects)

  plot_transect_signals(transects_signals, figure_path, filename)


In [4]:
import pandas as pd
from ast import literal_eval

scale_data = pd.read_csv('/content/drive/MyDrive/Images/SalmonFishScalesCoordinates-V2.csv', sep=",",
                          names=['filename', 'lt_center', 'br_center', 'end_point'],
                          dtype={'filename': 'string'},
                          converters={'lt_center': literal_eval,
                                      'br_center': literal_eval,
                                      'end_point': literal_eval},
                          header=0)
scale_data['filepath'] = scale_data.apply(lambda row: os.path.join(PATH, row['filename']) +  ".tif", axis=1)

# Switch from column-major to row major
scale_data['lt_center'] = scale_data.apply(lambda row: (row['lt_center'][1], row['lt_center'][0]) , axis = 1)
scale_data['br_center'] = scale_data.apply(lambda row: (row['br_center'][1], row['br_center'][0]) , axis = 1)
scale_data['end_point'] = scale_data.apply(lambda row: (row['end_point'][1], row['end_point'][0]) , axis = 1)

scale_data['mid_center'] = scale_data.apply(lambda row: (int((row['lt_center'][0] + row['br_center'][0]) / 2),
                                                         int((row['lt_center'][1] + row['br_center'][1]) / 2)) , axis=1)
scale_data['transect_angle'] = scale_data.apply(lambda row: calculate_transect_angle(row['mid_center'], row['end_point']), axis=1)

scale_data['id'] = scale_data.apply(lambda row: int(row['filename'].split('_')[0]), axis=1)

# Bring in labels
label_data = pd.read_csv('/content/drive/MyDrive/Images/labels.csv', sep=",",
                          names=['id', 'age', 'type', 'fold'],
                          dtype={'filename': 'int64', 'age': 'int32', 'type': 'int32', 'fold': 'int32'},
                          header=0)

# Fold in labels with scale data
scale_data = scale_data.merge(label_data, on='id', how='left', validate='many_to_one')

# Drop any rows with NaNs, recast ints
scale_data = scale_data.dropna().astype({'fold': int, 'type': int})

In [5]:
# Get transect signals
transects_signals = []

for index, row in scale_data.iterrows():
  #print(index, row['id'], row['filename'], row['fold'])

  gray = load_img_to_gray(row['filepath'])
  transects = calculate_coordinates_of_transects(row['mid_center'],
                                                 row['transect_angle'],
                                                 .1, 11)

  transects_signals.append([row['id'],
                           extract_transects_signals(gray, transects)],
                           )

transects_signals_df = pd.DataFrame(transects_signals, columns=['id', 'signals'])

# Merge with labels and explode
transects_signals_df = transects_signals_df.merge(label_data, on='id',
                                                  how='left',
                                                  validate='many_to_one')\
                                                  .explode('signals', ignore_index=True)\
                                                  .rename(columns={"signals": "signal"})
transects_signals_df['signal'] = transects_signals_df['signal'].apply(lambda x: np.array(x))

# Add convolved signals
transects_signals_df['convolved_signal'] = transects_signals_df.apply(lambda row: convolve_transects_signal(row['signal'], 5), axis=1)

## K-fold Cross validation

In [14]:
# Create a copy of transects_signals_df so that it doesn't have to be generated
#  each time
from numpy.fft import fft, ifft
from sklearn.model_selection import StratifiedKFold
import keras

def rolling_window(a, window, step):
    shape = a.shape[:-1] + ((a.shape[-1] - window + 1)//step, window)
    strides = (a.strides[0] * step,) + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

df = transects_signals_df.copy(deep=True)
df['signal'] = df['signal'].apply(lambda x: np.pad(x, (0, int(1.4 * TRANSECT_LENGTH) - x.size), 'constant', constant_values=(0, 0)))
df['signal'] = df['signal'].apply(lambda x: np.flip(x))
df['convolved_signal'] = df['convolved_signal'].apply(lambda x: np.pad(x, (0, int(1.4 * TRANSECT_LENGTH) - x.size), 'constant', constant_values=(0, 0)))
df['convolved_signal'] = df['convolved_signal'].apply(lambda x: np.flip(x))
df['convolved_freq'] = df['signal'].apply(lambda x: np.abs(fft(x))[0:75])

# Cross-validation
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Iterate over each fold for cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(df['convolved_freq'], df['type'])):
    print(f"Fold {fold + 1}/{NUM_FOLDS}")

    # Split the data into training and validation sets for this fold
    x_train, x_val = df['convolved_freq'][train_index], df['convolved_freq'][val_index]
    y_train, y_val = df['type'][train_index], df['type'][val_index]

    # Normalize the data
    x_train = np.stack(x_train)
    x_max = np.max(x_train, axis=1, keepdims=True)
    x_train = x_train / x_max
    x_train = x_train.reshape(x_train.shape[0], -1)  # Flatten the array

    # Normalize validation data using the maximum value of the training data
    x_val = np.stack(x_val)
    x_val = x_val / np.max(x_train)

    # Create and compile the model for this fold
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dropout(.5),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dropout(.75),
        keras.layers.Dense(124, activation='relu'),
        keras.layers.Dropout(.75),
        keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(x=x_train, y=y_train, batch_size=256, epochs=350, validation_data=(x_val, y_val))


Streaming output truncated to the last 5000 lines.
Epoch 305/350
9/9 [==============================] - 0s 23ms/step - loss: 0.6640 - accuracy: 0.6072 - val_loss: 14098.7412 - val_accuracy: 0.6186
Epoch 306/350
9/9 [==============================] - 0s 23ms/step - loss: 0.6573 - accuracy: 0.6133 - val_loss: 15641.6074 - val_accuracy: 0.6229
Epoch 307/350
9/9 [==============================] - 0s 23ms/step - loss: 0.6592 - accuracy: 0.6128 - val_loss: 16576.4414 - val_accuracy: 0.6229
Epoch 308/350
9/9 [==============================] - 0s 23ms/step - loss: 0.6594 - accuracy: 0.6081 - val_loss: 16401.8320 - val_accuracy: 0.6102
Epoch 309/350
9/9 [==============================] - 0s 24ms/step - loss: 0.6547 - accuracy: 0.6180 - val_loss: 14873.3486 - val_accuracy: 0.6144
Epoch 310/350
9/9 [==============================] - 0s 21ms/step - loss: 0.6556 - accuracy: 0.6143 - val_loss: 14957.1748 - val_accuracy: 0.6144
Epoch 311/350
9/9 [==============================] - 0s 20ms/step - loss: